In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import asyncio
import json
import random
import re
from typing import TypedDict

from dotenv import load_dotenv

import art
from art.local import LocalBackend

load_dotenv()


class TemporalCluePuzzle(TypedDict):
    num_clues: int
    prompt: str
    solution: dict[str, str]


puzzles_path = "../data/temporal-clue/puzzles.json"
puzzles: list[TemporalCluePuzzle] = json.loads(open(puzzles_path).read())
val_puzzles = puzzles[:64]
test_puzzles = puzzles[64:128]
train_puzzles = puzzles[128:]
random.seed(42)
random.shuffle(train_puzzles)


async def get_prompt_logprobs(
    model: art.Model, puzzle: TemporalCluePuzzle
) -> art.Trajectory:
    messages: art.Messages = [
        {"role": "user", "content": puzzle["prompt"] + " /no_think"}
    ]
    client = model.openai_client()
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name, max_tokens=4096
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    num_correct = 0
    for key, value in puzzle["solution"].items():
        if matches := re.findall(rf"{key}\. ([A-Za-z \.:-]+)", content):
            match = matches[-1]
            if match.strip().lower() == value.lower():
                num_correct += 1
    reward = acc = num_correct / len(puzzle["solution"])
    return art.Trajectory(
        messages_and_choices=[*messages, choice], reward=reward, metrics={"acc": acc}
    )


model = art.TrainableModel(
    name="066",
    project="temporal-clue",
    base_model="Qwen/Qwen2.5-7B-Instruct",
    _internal_config=art.dev.InternalModelConfig(
        _decouple_vllm_and_unsloth=True,
        engine_args=art.dev.EngineArgs(max_logprobs=1, gpu_memory_utilization=0.8),
    ),
)
backend = LocalBackend()
await model.register(backend)

stride = 8
for i in range(await model.get_step(), 1_000):
    val_groups, train_groups = await asyncio.gather(
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(
                    get_prompt_logprobs(model, puzzle) for _ in range(1)
                )
                for puzzle in val_puzzles
            ),
            pbar_desc="val",
            pbar_total_completion_tokens=False,
        ),
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(
                    get_prompt_logprobs(model, puzzle) for _ in range(16)
                )
                for puzzle in train_puzzles[i * stride : (i + 1) * stride]
            ),
            pbar_desc="train",
            pbar_total_completion_tokens=False,
        ),
    )
    for group in train_groups:
        max_reward = max(trajectory.reward for trajectory in group)
        for trajectory in group:
            trajectory.metrics["max_reward"] = max_reward
    break
    await model.log(val_groups)
    await model.delete_checkpoints()
    await model.train(
        train_groups,
        config=art.TrainConfig(learning_rate=5e-6),
        _config=art.dev.TrainConfig(precalculate_logprobs=True, scale_rewards=False),
    )

wandb: Currently logged in as: bhilton (wandb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


INFO 10-16 19:47:35 [__init__.py:235] Automatically detected platform cuda.


/home/sky/sky_workdir/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore # noqa: F401


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-16 19:47:41 [__init__.py:235] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.10.0.
   \\   /|    NVIDIA H200. Num GPUs = 2. Max memory: 139.811 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]
Unsloth 2025.10.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


...

INFO 10-16 19:48:17 [default_loader.py:262] Loading weights took 2.29 seconds
INFO 10-16 19:48:17 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 10-16 19:48:17 [gpu_model_runner.py:1892] Model loading took 14.4647 GiB and 2.976039 seconds
INFO 10-16 19:48:24 [backends.py:530] Using cache directory: /home/sky/.cache/vllm/torch_compile_cache/28fac1a21f/rank_0_0/backbone for vLLM's torch.compile
INFO 10-16 19:48:24 [backends.py:541] Dynamo bytecode transform time: 6.46 s
INFO 10-16 19:48:29 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 4.234 s
INFO 10-16 19:48:30 [monitor.py:34] torch.compile takes 6.46 s in total
INFO 10-16 19:48:31 [gpu_worker.py:255] Available KV cache memory: 96.58 GiB
INFO 10-16 19:48:31 [kv_cache_utils.py:833] GPU KV cache size: 1,808,448 tokens
INFO 10-16 19:48:31 [kv_cache_utils.py:837] Maximum concurrency for 32,768 tokens per request: 55.19x


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:05<00:00,  6.64it/s]


INFO 10-16 19:48:37 [gpu_model_runner.py:2485] Graph capturing finished in 6 secs, took 0.51 GiB
INFO 10-16 19:48:37 [core.py:193] init engine (profile, create kv cache, warmup model) took 19.69 seconds
INFO 10-16 19:48:38 [loggers.py:141] Engine 000: vllm cache_config_info with initialization after num_gpu_blocks is: 113028


val:   0%|          | 0/64 [00:00<?, ?it/s]

train:   0%|          | 0/128 [00:00<?, ?it/s]

In [38]:
from openai.types.chat.chat_completion import ChatCompletion


def students_and_teachers(
    group: art.TrajectoryGroup,
) -> list[tuple[art.Trajectory, art.Trajectory]]:
    trajectories = group.trajectories.copy()
    random.shuffle(trajectories)
    return list(zip(trajectories, trajectories[1:] + trajectories[:1]))


async def get_conditioned_logprobs(
    group: art.TrajectoryGroup, student: art.Trajectory, teacher: art.Trajectory
) -> tuple[art.TrajectoryGroup, art.Trajectory, art.Trajectory, ChatCompletion]:
    return (
        group,
        student,
        teacher,
        await model.openai_client().chat.completions.create(
            messages=teacher.messages() + student.messages(),
            model=model.name,
            max_tokens=1,
            extra_body={"prompt_logprobs": 1, "return_tokens_as_token_ids": True},
        ),
    )


results = await asyncio.gather(
    *(
        get_conditioned_logprobs(group, student, teacher)
        for group in train_groups
        for student, teacher in students_and_teachers(group)
    )
)

In [39]:
group, student, teacher, chat_completion = results[0]

In [40]:
student.reward, teacher.reward

(0.4, 0.2)

In [41]:
[
    max(token_logprobs.items(), key=lambda x: x[1]["rank"])
    for token_logprobs in chat_completion.prompt_logprobs
    if token_logprobs is not None
]

[('8948',
  {'logprob': -14.043458938598633, 'rank': 28892, 'decoded_token': 'system'}),
 ('198', {'logprob': -2.4143319129943848, 'rank': 1, 'decoded_token': '\n'}),
 ('2610',
  {'logprob': -10.547307968139648, 'rank': 3452, 'decoded_token': 'You'}),
 ('525',
  {'logprob': -0.24837584793567657, 'rank': 1, 'decoded_token': ' are'}),
 ('1207', {'logprob': -10.388444900512695, 'rank': 77, 'decoded_token': ' Q'}),
 ('16948',
  {'logprob': -0.0066816010512411594, 'rank': 1, 'decoded_token': 'wen'}),
 ('11', {'logprob': -0.004796546418219805, 'rank': 1, 'decoded_token': ','}),
 ('3465',
  {'logprob': -11.95403003692627, 'rank': 31, 'decoded_token': ' created'}),
 ('553',
  {'logprob': -0.0007244108128361404, 'rank': 1, 'decoded_token': ' by'}),
 ('54364',
  {'logprob': -0.001134824356995523, 'rank': 1, 'decoded_token': ' Alibaba'}),
 ('14817',
  {'logprob': -0.0029226704500615597, 'rank': 1, 'decoded_token': ' Cloud'}),
 ('13', {'logprob': -0.005659153219312429, 'rank': 1, 'decoded_token': 

In [23]:
print(
    "".join(
        max(token_logprobs.items(), key=lambda x: x[1]["rank"])[1]["decoded_token"]
        for token_logprobs in chat_completions[0].prompt_logprobs
        if token_logprobs is not None
    )
)

system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
On a dark winter night, wealthy and enigmatic Mr. John Q. Boddy hosted a small, but lavish, dinner party for some of his closest associates. However, the night ended in tragedy when Mr. Boddy was found dead in one of the rooms of Tudor Mansion in the early hours of the morning. The following persons of interest have been identified as suspects:

• Mrs. White
• Madame Rose
• Sgt. Gray
• Monsieur Brunette
• Mrs. Peacock
• Miss Scarlet
• Professor Plum
• Miss Peach
• Colonel Mustard

And the following weapons were found on the premises:

• Lead Pipe
• Poison
• Knife
• Candlestick
• Horseshoe
• Rope

The murder could only have occured in one of the following rooms:

01. Cloak Room
02. Billiard Room
03. Conservatory
04. Trophy Room
05. Studio
06. Courtyard
07. Fountain
08. Drawing Room
09. Carriage House
10. Lounge

The rooms are laid out as follows:

  NN NN NN  
W 01|02|03 E
W 04|05|06 E